In [3]:
from velib_ml.data import load_timeseries
...


In [14]:
from pathlib import Path
import json, pandas as pd
from velib_ml.data import load_timeseries
from velib_ml.features import make_features
from velib_ml.splits import split_train_test
from velib_ml.config import SPLIT_TRAINTEST

ROOT = Path.cwd().parent                 # repo root
DATA = ROOT / "data" / "raw" / "velib_timeseries_5min.csv"
ART  = ROOT / "artifacts" / "v0_1"

# 1) Load raw and build features (same toggles as training)
df   = load_timeseries(str(DATA))
feat = make_features(df, use_ema=False)   # you trained with --no-ema
for d in (feat,):                         # add occ_now like at training
    d["occ_now"] = d["occ"].astype("float32")

# 2) Train/test split (same quantile as training), then pick a valid row
train, test = split_train_test(feat, SPLIT_TRAINTEST)

feat_cols = json.load(open(ART / "feat_cols_delta.json"))
row = test[feat_cols].dropna().head(1)    # one fully-defined row

# 3) Save to repo root (so your curl/requests can reuse it)
out = ROOT / "sample_features.csv"
row.to_csv(out, index=False)
print("Wrote:", out, "with", row.shape[1], "columns")


/Users/sako/Documents/PROJET DISPO VELIB/src/velib_ml/data.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df["capacity"] = (df.groupby("station_id")["capacity"]
/Users/sako/Documents/PROJET DISPO VELIB/src/velib_ml/features.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  def shift(col,k): return feat.groupby("station_id")[col].shift(k)
/Users/sako/Documents/PROJET DISPO VELIB/src/velib_ml/features.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the fut

Wrote: /Users/sako/Documents/PROJET DISPO VELIB/sample_features.csv with 22 columns


In [19]:
import pandas as pd, requests, json, pathlib
ROOT = pathlib.Path.cwd().parent
row = pd.read_csv(ROOT/"sample_features.csv").iloc[0].fillna(0.0).to_dict()
r = requests.post("http://127.0.0.1:8000/predict/30", json={"features": row})
print(r.json())


{'y_hat_bikes': 5.0, 'occ_hat': 0.1852}
